# Notebook 1: Introduction to POML

**Prompt Orchestration Markup Language**

Based on:
- https://betterstack.com/community/guides/ai/poml-markup/
- https://microsoft.github.io/poml/stable/

## Learning Objectives
- Understand why structured prompts matter for maintainability
- Write basic POML prompts using core tags
- Use POML templates with variables and Python context

## 1. Setup

First, let's install the required packages and set up our environment.

In [ ]:
# Install required packages
!pip install poml langchain==1.2.7 langchain-groq python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
from poml import poml

# Load environment variables (for API keys later)
load_dotenv()

# Set up Groq API key
if not os.getenv('GROQ_API_KEY'):
    os.environ['GROQ_API_KEY'] = input('Enter your Groq API key: ')

## 2. Why POML?

### The Problem with Plain Text Prompts

As AI applications become more sophisticated, prompts can quickly become:
- **Hard to read**: Long strings with instructions, examples, and data mixed together
- **Hard to maintain**: Changes require finding and updating text in multiple places
- **Hard to reuse**: Copy-pasting leads to inconsistencies

### Example: A Standard Prompt

In [ ]:
# This is how prompts often look in real applications - messy!
standard_prompt = """
You are an expert at explaining complex topics.
Explain this machine learning.
Aim for an advanced level of explanation depth.
"""

print(standard_prompt)

Issues with this are that all the information is hardcoded into the prompt and that all types of information are mixe together.

### POML Solution

POML (Prompt Orchestration Markup Language) brings structure to prompts using an HTML-like syntax with semantic tags:

| Tag | Purpose |
|-----|--------|
| `<role>` | Define the AI's persona/system message |
| `<task>` | Specify the main objective |
| `<hint>` | Provide additional guidance |
| `<example>` | Include few-shot examples |

## 3. POML Basics

### Your First POML Prompt

Let's rewrite that messy prompt using POML:

In [ ]:
poml_prompt = """
<poml>
  <role>You are an expert at explaining complex topics.</role>
  <task>Explain this topic: {{topic}}</task>
  <hint>Aim for this level of explanation depth: {{explanation_depth}}</hint>
</poml>
"""

# The context is a dictionary passed to the poml function
# The keys in the dictionary become available as variables inside the .poml file
context = {
    'topic': 'machine_learning',
    'explanation_depth': 'advanced'
}

output = poml(poml_prompt, context)
print(output[0]['content'])

### Key Benefits

1. **Structure is visible**: Tags clearly separate role, task, hints, and content
2. **Self-documenting**: The markup explains what each part does
3. **Easy to modify**: Change one section without affecting others

### Compiling to Different Formats

POML can output different formats using the `syntax` attribute:

Supported formats: markdown, html, json, yaml, xml, text

In [ ]:
# Compile to JSON format
json_prompt = """
<poml syntax="json">
  <role>You are a code reviewer.</role>
  <task>Review the provided code for bugs.</task>
  <hint>Focus on logic errors, not style.</hint>
</poml>
"""

result = poml(json_prompt)
print("JSON output:")
print(result[0]['content'])

## 4. Templates and Variables

POML's real power comes from its template engine. You can use:
- **Variables**: `{{variable_name}}` for dynamic content
- **`<let>`**: Define variables within the template
- **`if`**: Conditional content
- **`for`**: Loop over lists

### Using Variables with Python Context

In [ ]:
# Define a template with variables
template_prompt = """
<poml>
  <role>You are a helpful {{role_type}} assistant.</role>
  <task>Explain {{topic}} to a {{audience}} audience.</task>
  <hint>Keep it {{style}}.</hint>
</poml>
"""

# Pass context from Python
context = {
    "role_type": "technical",
    "topic": "neural networks",
    "audience": "beginner",
    "style": "concise with examples"
}

result = poml(template_prompt, context)
print("Dynamic prompt:")
print(result[0]['content'])

In [ ]:
# Try with different context - same template, different output!
context_advanced = {
    "role_type": "academic",
    "topic": "transformer architecture",
    "audience": "graduate student",
    "style": "detailed with mathematical notation"
}

result = poml(template_prompt, context_advanced)
print("Same template, different context:")
print(result[0]['content'])

### Conditionals with `if`

In [ ]:
# Conditional content based on context
conditional_prompt = """
<poml>
  <role>You are a helpful assistant.</role>
  <task>Answer the user's question about {{topic}}.</task>
  
  <hint if="include_examples">Include 2-3 concrete examples.</hint>
  <hint if="keep_short">Keep your response under 100 words.</hint>
</poml>
"""

# Context with examples enabled, short disabled
context = {
    "topic": "Python loops",
    "include_examples": True,
    "keep_short": False
}

result = poml(conditional_prompt, context)
print("With examples, no length limit:")
print(result[0]['content'])
print("\n" + "="*50 + "\n")


# Now flip the conditions
context["include_examples"] = False
context["keep_short"] = True

result = poml(conditional_prompt, context)
print("No examples, keep short:")
print(result[0]['content'])

### For loops

In [ ]:
# Loop over a list of items
loop_prompt = """
<poml>
  <role>You are a quiz generator.</role>
  <task>Create a multiple choice question for each of the following topics:</task>
  
  <list>
    <item for="topic in topics">{{topic}}</item>
  </list>
</poml>
"""

context = {
    "topics": ["Machine Learning", "Neural Networks", "Natural Language Processing"]
}

result = poml(loop_prompt, context)
print("Loop output:")
print(result[0]['content'])

## 5. Using POML with LangChain and Groq

Let's put it all together and actually call an LLM with our POML prompt!

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage

# Initialize the Groq LLM
llm = ChatGroq(model="openai/gpt-oss-20b", temperature=0.7)

# Create a POML prompt
explanation_prompt = """
<poml>
  <role>You are a friendly teacher who excels at explaining complex topics simply.</role>
  <task>Explain {{topic}} in 2-3 sentences.</task>
  <hint>Use an analogy if it helps.</hint>
</poml>
"""

# Compile with context
context = {"topic": "how neural networks learn"}
compiled = poml(explanation_prompt, context)

# Send to the LLM
response = llm.invoke([HumanMessage(content=compiled[0]['content'])])
print("LLM Response:")
print(response.content)

## 6. Quick Exercise

**Your turn!** Create a POML prompt that:
1. Takes a `topic` variable
2. Has a role as an "expert educator"
3. Tasks the AI with creating a brief explanation
4. Includes a conditional hint for `include_quiz` that adds "End with a simple quiz question"

Test it with `topic="recursion"` and `include_quiz=True`

In [ ]:
# YOUR CODE HERE
# Create your POML prompt template
my_prompt = """
<poml>
  <!-- Add your role, task, and conditional hint here -->
</poml>
"""

# Define context
my_context = {
    "topic": "recursion",
    "include_quiz": True
}

# Compile and print
result = poml(my_prompt, my_context)
print(result[0]['content'])

# Send to LLM
response = llm.invoke([HumanMessage(content=result[0]['content'])])
print(response.content)

## Summary

In this notebook, you learned:

1. **Why POML matters**: Structured prompts are more maintainable and reusable
2. **Core tags**: `<role>`, `<task>`, `<hint>` for semantic structure
3. **Templates**: Variables (`{{}}`), conditionals (`if`), and loops (`for`)
4. **Python integration**: Passing context dictionaries to POML templates